1. 为什么缩放点积attention要除以根号d，为什么是根号d？
当q、k维度较高时点积结果可能较大，引起softmax梯度消失，所以除以根号d缩放；
q、k的点积方差为d，除以根号d能将点积方差稳定到1，有利于模型训练的稳定性。
1.1 为什么q、k的点积方差是d
假设q、k是独立的并且每维符合均值0方差1的分布，q、k的点积是均值0方差d
1.2 为什么点积结果大softmax梯度消失
如果softmax的输入存在某个过大的值，softmax的结果会是one-hot（因为指数函数斜率随着值增大而增大，使得x轴上的小变化引起y轴的大变化）
softmax对每个输入求导1）i=j，y'=pi * (1-pi) 2)i!=j, y'= -pi * pj 都会是接近零的数
1.3 为什么点积能衡量向量的相似性
归一化后的两个向量a、b的欧式距离的平方与a、b点积负相关：d(a,b)^2 = 1 - 2ab

2. 为什么要多头，不单头
多个子空间信息，每个头的参数随机初始化并且各自独立学习，给模型学到不同的注意力分布提供了条件（相当于CNN中的多通道）
self attention会让每个位置的词学到和自身最大的注意力，多头能获取多种注意力的分配方式
原本的高维空间向量的分布稀疏，而attention通过点击计算相似性，高维空间不利于模型学习向量间的相似性，分割多头让原本高维空间分割为多个低维空间
从transformer的实验结果看，不是头越多越好或者越少越好

3. 为什么qkv要经过原始词向量分别用三个线性变换的到？不用原始的词向量
如果不经单独的线性变换，attention score矩阵会是对称矩阵，qk经过不同的投影增强attention score矩阵的泛化能力
如果不经投影，每个词互相的注意力分数一样。“我是一个男孩”中“男孩”对“我”对修饰性应该大雨“我”对“男孩”的修饰性。

4. 为什么attention score用缩放点积，不用加法（MLP）？
计算复杂度低，加法的隐藏层数一般是d层，复杂度为O(nndd)
在维度低时点积和加法表现差不多，在dk升高时加法注意力的表现优势更明显

5. 在计算attention时，如何对padding做mask
在要mask的位置，对点积的结果加一个负无穷

6. 训练和预测时，decoder的输入
训练：第一个self attention的QKV一样，是整个序列，在算注意力矩阵时，用mask矩阵只激活这个词之前的注意力
推理：第一个self attention的Q是上一个推理出来的词，K、V是已经预测出来的序列

7. 残差连接的作用
残差连接是让输入层跳过中间多层，直接加在输出层的方法。这样缩短输入层到输出层的路径，让输出层的梯度能直达输入层，缓解深层网络带来梯度消失的问题，同时加快模型的收敛；另一方面增强模型的表达能力，因为输出层能直接获取原始信息，避免原始信息在多层非线性变换中丢失。

8. 层归一化的作用，为什么不用批量归一化
归一化能稳定数值分布，加快模型收敛。
transformer的输入是可变长度的序列，如果使用批量归一化，padding会对同一时间步真实的词向量造成干扰。
使用层归一化，在每个样本内独立归一化，不受序列长度影响。

9. 介绍transformer？transformer是一种基于自注意力机制的神经网络结构，避开传统CNN、RNN处理序列问题上的局限性，实现高效并行计算和良好的长序列建模能力。其核心思想是自注意力机制，通过计算序列中每个元素和其他元素的注意力得分，动态聚合全局关系。

10. transformer的结构是怎样的？transformer架构上是encoder-decoder结构，encoder、decoder由各自的大模块堆叠而成。
- encoder的大模块有两个子模块多头自注意力和前馈神经网络，每个子模块后接残差链接和层归一化。除了第一个模块的输入是经过位置编码的原始序列，每个模块的输入是上一个模块的输出，最后一个模块的输出是encoder的输出。
- decoder的大模块包含三个子模块，掩码多头自注意力、多头交叉注意力、前馈神经网络，每个子模块后接残差链接和层归一化。第一个掩码多头自注意力的输入在训练和预测时不同，训练时输入整个序列，用掩码控制每个词只能看到他之前的词，QKV是一样的，执行自注意力的计算，预测时Q为上一个预测的词，KV是已经生成的序列。第二个多头的Q是decoder第一个多头的输出，KV是encoder的输出。

    10.1 为什么第二个多头的Q是decoder序列，KV是encoder序列？因为attention本质上是对Q的编码，decoder的第二个attention是对目标序列编码，所以要用第一个多头的输出作为Q。
    
        为什么attention的本质是对Q编码？因为QKV中KV是一一映射的，两者是同一事物的两个方面，类似于谜面和谜底。Q计算和序列中K的注意力得分，再从序列的V中动态聚合信息，所谓的动态聚合就是对V序列做加权平均。从数学形式上看，如果不考虑缩放根号dk和softmax，attention是Q*K转置*V，输出矩阵的行数是Q的长度。

11. 为什么FFN比自注意力更重要？ 因为自注意力（建模元素间的全局关系）是线性变换，没有非线性能力。FFN引入非线性，让模型能够学习更复杂的表示，用两个全连接层对特征进一步处理（非线性的季或函数+特征扩张和压缩），增强模型能力。在对比移除自注意力和FFN的对比实验中，移除自注意力对模型性能影响较小，FFN导致性能显著下降。（在参数量上FFN比多头自注意力层多一倍：FFN是512\*2048+2048\*512，多头是512\*512\*3+512\*512,FFN/多头=2：1）

12. self-attention为什么要使用Q、K、V，仅仅使用Q、V/K、V或者V为什么不行？因为通过计算QK的相似度，能获取每个元素与其他每个元素的关系，这是attention的内在要求。仅有QV或者KV缺少了计算注意力的步骤。通过结合attention score矩阵和V，实现动态聚合全局信息。

13. 为什么说Transformer可以代替seq2seq？并行、长序列建模（基于rnn的seq2seq encoder将不同长度的序列信息压缩到相同长度的向量作为decoder获取输入序列信息的方式，对长序列不公平；rnn的序列建模是逐个计算的，这导致当两个词之间的距离变长，两者之间信息上的依赖关系变弱，而attention能直接获取任意两个词的关系。）

14. 如何加入词序信息的？对词向量的不同维度使用周期不同的正余弦函数，周期随着维度索引指数增加，低维度的特征周期小，携带局部的位置信息，高维度特征携带全局位置关系，随着周期变大，位置编码能捕获更全局的位置关系，不同时间步的词的索引作为正余弦函数的输入生成与词向量相同维度的位置编码，将位置编码和词向量相加，将位置信息注入到序列中

https://zhuanlan.zhihu.com/p/82391768


https://www.nowcoder.com/discuss/387725948110602240

证明点积的方差是维度数

In [ ]:
import numpy as np 
arr1=np.random.normal(size=(3,1000))
arr2=np.random.normal(size=(3,1000))
result=np.dot(arr1.T,arr2) # 这里是 arr1.T
arr_var=np.var(result)
print(arr_var)


演示对padding的mask

In [2]:
import tensorflow as tf
import numpy as np

In [19]:
batch_size = 4
seq_len = 8
logits = tf.random.normal([batch_size, seq_len, seq_len])
valid_len = tf.range(start=2, limit=6)
valid_len, logits.shape

(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([2, 3, 4, 5], dtype=int32)>,
 TensorShape([4, 8, 8]))

In [20]:
def softmax_mask(logits, valid_len):
    range_matrix = tf.range(seq_len)[None, :] # 1, seq_len
    valid_len = valid_len[:, None] # batchsize, 1
    mask = range_matrix < valid_len # batch_size, seq_len
    mask = mask[:, tf.newaxis, :]
    mask = tf.logical_and(mask, tf.transpose(mask, perm=[0,2,1]))
    print(mask[0])
    mask = tf.cast(mask, tf.float32)
    masked_logits = logits * mask + (1-mask)*-1e-9
    return masked_logits
softmax_mask(logits=logits, valid_len=valid_len)

tf.Tensor(
[[ True  True False False False False False False]
 [ True  True False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]], shape=(8, 8), dtype=bool)


<tf.Tensor: shape=(4, 8, 8), dtype=float32, numpy=
array([[[ 5.3475684e-01, -2.2182615e+00, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-1.5040651e-01,  1.1526464e+00, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10,
         -9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997e-10],
        [-9.9999997e-10, -9.9999997e-10, -9.9999997e-10, -9.9999997